# Creating a Gridded Library in WebbPSF with psf_grid()

The goal was to create this method to be as similar to calc_psf() as possible for those people who already use WebbPSF (also because it then lies within the WebbPSF structure very nicely).

So certain parameters are defined before the method call, and certain parameters are defined within the method call.

Below is a summary:

## Summary of Possible Parameters

    Parameters to be set before psf_grid()
    --------------------------------------
    filter(s)
        Set with: inst.filter = str
        If you want to run all the filters - must loop through them
    detector(s)
        Set with: inst.detector = str
        If you want to run 1 detector - define it like above and set the internal parameter all_detectors=False
        If you want to run a list of detectors - must loop through them and set all_detectors=False
        If you want to run all the detectors - set all_detectors=True
    psf location if num_psfs=1 
        Set with: inst.detector_position = tuple in the form (x,y)
        Or if you want the location to be the center of the detector, set single_psf_centered=True
    anything else from WebbPSF
        E.g. inst.pupil_mask, inst.image_mask, inst.pupilopd, etc
    
    
    
    Parameters inside psf_grid()
    ----------------------------
    all_detectors: bool
        If True, run all detectors for the instrument. If False, run for the detector set in
        the instance. Default is True
    use_detsampled_psf: bool
        If True, the grid of PSFs returned will be detector sampled (made by binning down the
        oversampled PSF). If False, the PSFs will be oversampled by the factor defined by the
        oversample/detector_oversample/fft_oversample keywords. Default is False.
    num_psfs: int
        The total number of fiducial PSFs to be created and saved in the files. This
        number must be a square number. Default is 16.
        E.g. num_psfs = 16 will create a 4x4 grid of fiducial PSFs.
    single_psf_centered: bool
        If num_psfs is set to 1, this defines where that psf is located. If True it will be the
        center of the detector, if False it will be the location defined in the WebbPSF
        attribute detector_position (reminder - detector_position is (x,y))
    save: bool
        True/False boolean if you want to save your file. Default is False.
    outfile: str
        If "save" keyword is set to True, your current file will be saved under
        "{outfile}_det_filt.fits". Default of None will save it in the current
        directory as: instr_det_filt_fovp#_samp#_npsf#.fits
    overwrite : bool
        True/False boolean to overwrite the output file if it already exists. Default
        is True.
    **kwargs:
        Add any extra arguments to the WebbPSF calc_psf() method call. This includes
        the following kwargs (and their default values): source(=None), nlambda(=None),
        monochromatic(=None), fov_arcsec(=None), fov_pixels(=101), oversample(=4),
        detector_oversample(=None), fft_oversample(=None), normalize(='first'),
        add_distortion(=True), and crop_psf(=True)


## Example Uses

In [ ]:
import webbpsf

### Create a Full Library for 1 Filter and All Detectors

This has all_detectors default to True because we expect this to be the normal use case

In [ ]:
nir = webbpsf.NIRCam()
nir.filter = "F090W"
nir.psf_grid(all_detectors=True)


Starting filter: F090W
  Running detector: NRCA1
  Running detector: NRCA2


### Create a Library for 1 Filter and 1 Detector

In [ ]:
nir = webbpsf.NIRCam()
nir.filter = "F090W"
nir.detector = "NRCA2"
nir.psf_grid(all_detectors=False)

### Want multiple filters? Will loop through them

In [ ]:
nir = webbpsf.NIRCam()
for filt in nir.filter_list:
    nir.filter = filt
    nir.psf_grid(all_detectors=True)

### Set the number of PSFs

Use num_psfs to set the size of the grid. num_psfs = 4 means you'll have a 2x2 grid. Default for now is 16

In [ ]:
nir = webbpsf.NIRCam()
nir.filter = "F090W"
nir.psf_grid(all_detectors=True, num_psfs=4)

### If num_psfs = 1 -> set the location of the PSF with detector_position attribute

If single_psf_centered = True, then the single PSF will be at the center of the detector. If it's False, then the location of the PSF is pulled from the detector_position attribute

I don't forsee this being of huge use, but it's good to cover the option

In [ ]:
nir = webbpsf.NIRCam()
nir.filter = "F090W"
nir.detector_position = (10,10) # (x,y)
nir.psf_grid(all_detectors=True, num_psfs=1, single_psf_centered=False)

### Setting other things before the psf_grid() method call

Anything you can set in WebbPSF can be applied here. For example, say you want to use a differend OPD, your own OPD with mirror moves in it, a pupil mask or an image mask, etc:

In [ ]:
nir = webbpsf.NIRCam()
nir.filter = "F090W"

# Setting extra things:
nir.pupilopd = ('OPD_RevW_ote_for_NIRCam_predicted.fits.gz', 7)
nir.pupil_mask = "WEAK LENS +4"

nir.psf_grid(all_detectors=True)

## Questions for The Group:

* File naming/saving -> What kind of options do we want people to have?
* Any case where users will want to use the detector-sampled PSF
* Default values (both in the CreatePSFLibrary class and the psf_grid method)

### File Saving/Naming

The main problem:
* In the case of NIRCam: when you run this method for all_detectors=True, it loops through a bunch of detectors, saving them one by one. So you can't pass 1 full output filename to the method because it would be saving multiple files as the same name (and thus overwritting them)
* So I've been thinking about different ways to let users save their files:
    * A default name with lots of information based on the inputs (filt, det, num_psfs, etc), but that requires a default location too: current directory?
    * Let people put in their own path and the start of the name they want, but the name will have qualifying information appended to the end to keep the different detector names seperate?

Right now it looks like:

In [ ]:
# Don't save
nir.psf_grid(save=False)

# Save with your own name
nir.psf_grid(save=True, outfile="/path/to/file/start_of_name") # start_of_name_{det}_{filt}.fits

# Save with the default name
nir.psf_grid(save=True, outfile=None) # {instr}_{det}_{filt}_fovp{#}_samp{#}_npsf{#}.fits

### Detector-sampled PSF

Is there a case where users would want a grid of detector-sampled PSFs? I currently have the base set up to include this, but I want to check if it'll be needed before I finish setting it up.

It would look like:

In [ ]:
nir = webbpsf.NIRCam()
nir.filter = "F090W"
nir.psf_grid(use_detsampled_psf=True)

### Default Values

Do we need to change any of these values?

Right now the default values are:

In [ ]:
all_detectors: True
use_detsampled_psf: False
num_psfs: 16
single_psf_centered: True
save: False
outfile: None # will save to current directory with default naming -instr_det_filt_fovp{}_samp{}_npsf{}.fits
overwrite : True

calc_psf kwargs:
source: None
nlambda: None
monochromatic: None
fov_arcsec: None
fov_pixels: 101
oversample: 4
detector_oversample: None
fft_oversample: None
normalize: 'first'
add_distortion: True
crop_psf: True

### Any other concerns?